In [1]:
include("../src/Utero.jl")
using .Utero 

In [2]:
ctx = ⬅Context()
ForwardBackward!(ctx, x -> 2.0*x + x, AddParams!(ctx, 7.0))

(21.0, Any[3.0])

In [3]:
# should be 12.8
ctx = ⬅Context()
ForwardBackward!(ctx, x -> x^2.0 + x^2.0, AddParams!(ctx, 3.2)) 

(20.480000000000004, Any[12.8])

In [4]:
# Gradient should be 56.21
ctx = Utero.⬅Context()
ForwardBackward!(ctx, w -> 2.0 * (((2.0*w + 12.0) - w) * w) + sin(w^2.0), AddParams!(ctx, 7.0))

(265.0462473472405, Any[56.20829561241092])

In [5]:
# should be approx. (27.53, 3.0428)
ctx = Utero.⬅Context()
x, y = AddParams!(ctx, 13.2, 89.0)
ForwardBackward!(ctx, (x, y) -> (cos(y)^3 * 4sin(x))^2 + cos(x*y)^2, x, y) 

(1.0748674472530375, Any[27.53002464314667, 3.0428042273213602])

In [6]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W[5:5]' * W[5:5], 3)

(0.13696959811254447, Any[sparse([5], [1], [0.7401880791056945], 30, 1)])

In [7]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 1)

In [8]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 1)

In [9]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> W .* 2, nothing)
@show size(W.val)
print(size(grads[1]))
print(grads[1])


size(W.val) = (2, 1)
(2, 1)

[2.0; 2.0;;]

In [10]:
W = rand(2,1)
@show ((W' * W) .* W)' * W

ctx = Utero.⬅Context()
W1, W2, W3, W4 = AddParams!(ctx, rand(2,1), rand(2,1), rand(2,1), rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> ((W1' * W2) .* W3)' * W4, nothing)
@show size(W.val)
print(size(grads[1]))
print(grads[1])

((W' * W) .* W)' * W = [0.23951010183558064;;]
used

ErrorException: type Array has no field val

In [11]:
ctx = Utero.⬅Context()
W1, W2, W3 = AddParams!(ctx, rand(30,30), rand(40,30), rand(30,40))

function model(X)
    return  ReLU(W3 * ReLU(W2 * ReLU(W1 * X)))
end

X = rand(1500, 30)
Y = mapslices(row -> cos.(row), X, dims=2) 

println(ctx.Params[1].val)
for (x, y) in zip(eachrow(X), eachrow(Y))
    println(MeanSquaredError(model(x), y).val)
    (out, grads) = ForwardBackward!(ctx, x -> MeanSquaredError(model(x), y), x)
    GradientStep!(0.0001, ctx.Params, grads)
    exit("")
end
println(ctx.Params[1].val)


[0.7890014628360704 0.9339836565076601 0.13298443401238735 0.07690484485350457 0.516428122224738 0.9673988288951721 0.529554981034385 0.3161805055687943 0.23344410796971093 0.33226828213500637 0.9383019858975467 0.12940441962325033 0.9121455996421979 0.23117520943974912 0.8867736031779662 0.9977777246121899 0.9322963243357186 0.8703831620633621 0.43786418788562853 0.8545751319677234 0.5925999035652165 0.3555498531183535 0.4683765043020175 0.06926689111318407 0.8086572168525215 0.5256154494539 0.9684179324409818 0.40382256202816313 0.052382862165794575 0.7948050440629009; 0.02035441758695944 0.5821411074951702 0.6217619871991859 0.23394461507944242 0.6692257444586968 0.5404979648810565 0.565617691408853 0.8599867394644319 0.5167481186312731 0.7548514674606358 0.1609053771880088 0.33586181559733996 0.6592568097599592 0.8124253052351764 0.9851070301651972 0.9383784655536144 0.7334563265183595 0.26222588233522104 0.7427627167812855 0.3533246121667789 0.032433361790509596 0.0108350826533581

 0.39666605989592574 0.3519308406969738 0.4671346193931658 0.7045349199029071 0.12342424876371194 0.5910488969313273 0.8454110705079338 0.9957212715023428; 0.8217843994604861 0.5728748993772476 0.9740263405239638 0.9251663211877666 0.024599110197140672 0.1859067425232167 0.3233140829641602 0.6400125797704291 0.5637888935247448 0.12482861842899773 0.2671187174046987 0.8906325272431672 0.9251799229893444 0.7972239110940651 0.11956916631446013 0.7047755368854683 0.7725673312955735 0.30209931419844427 0.43954148632277745 0.5282555467839158 0.2883402272931326 0.5462476528399777 0.5079243809093367 0.5969653892420463 0.18753247107381454 0.015040778519167186 0.23553713551982103 0.8739546740762533 0.3401954207138077 0.07390031639305539; 0.9760381492561061 0.24548913417279383 0.5654912198287708 0.5593473613706895 0.3629107117032673 0.9947027759202288 0.6858067575863404 0.7090727159400273 0.4271278238221067 0.5612335210284397 0.7533568948703903 0.4997351182920885 0.135306588953801 0.6765643211398

1.4509766218793872e8


MethodError: MethodError: Cannot `convert` an object of type String to an object of type Int32

Closest candidates are:
  convert(::Type{T}, !Matched::Ptr) where T<:Integer
   @ Base pointer.jl:23
  convert(::Type{T}, !Matched::T) where T<:Number
   @ Base number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  ...


In [12]:
using MLDatasets

TrainData = MNIST(:train)
ValidData = MNIST(:test)

ctx = ⬅Context()

densenet = Sequential(Dense(28*28, 16, ReLU), Dense(16, 16, ReLU), Dense(16, 10, ReLU), Softmax)
#display(densenet(rand(28*28, 1)))
#display(MeanSquaredError(densenet(rand(28*28, 1)), rand(10, 1)))
AddParams!(ctx, densenet)
println(typeof(densenet.Sequence[1].W))
for (x, y) in DataIterator(TrainData.features, TrainData.targets)
    x = reshape(x, 28*28, 1)
    y = OneHot(Int64(y+1), 10)
    y = reshape(y, 10, 1)
    display(densenet.Sequence[1].W)
    (out, paramgrads) = ForwardBackward!(ctx, x -> MeanSquaredError(densenet(x), y), x)
    GradientStep!(0.0001, ctx.Params, paramgrads)
    display(paramgrads[1])
    display(densenet.Sequence[1].W)
    exit("")
end

Main.Utero.⬅Tracker{Matrix{

Main.Utero.⬅Tracker{Matrix{Float64}}([0.16701994294476297 -0.3069769695931129 … 0.5391368599198317 -0.08299321577163768; -0.17610219862843093 0.7389283799362942 … -0.5958143711963643 -0.23513485074741952; … ; -0.543307368478414 0.13842321869892216 … -1.4329373376591468 -1.0809672810768962; 0.02387335571676461 0.27217237089127777 … 1.5748724017382032 -0.14703267263256692], UUID("184faf7c-c545-11ee-1890-e7a2e9352cb8"), Any[], Any[])

Float64}}


usedsize(out.val) = (10, 1)


MethodError: MethodError: Cannot `convert` an object of type String to an object of type Int32

Closest candidates are:
  convert(::Type{T}, !Matched::Ptr) where T<:Integer
   @ Base pointer.jl:23
  convert(::Type{T1}, !Matched::CEnum.Cenum{T2}) where {T1<:Integer, T2<:Integer}
   @ CEnum ~/.julia/packages/CEnum/Bqafi/src/operators.jl:24
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number
   @ Base twiceprecision.jl:273
  ...
